In [2]:
# use CEASELESS Kernel on adrien local laptop

import os
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # => to add in Seastar env
import seastar as ss
from seastar.utils.tools import dotdict

from scipy import interpolate, stats # => to add in Seastar env ?
from scipy.optimize import least_squares

# from seastar.gmfs.doppler import *
import pdb

In [3]:
level1 = xr.Dataset(
    data_vars=dict(
            CentralWavenumber=( [],270 ),
            CentralFreq=( [], 13.5 * 10**9 ),
            IncidenceAngleImage=( ['across','along','Antenna'], np.full([5,6,4], 30) ),
            AntennaAzimuthImage=(['across', 'along', 'Antenna'],
                           np.stack((np.full([5, 6], 45),
                                     np.full([5, 6], 90),
                                     np.full([5, 6], 90),
                                     np.full([5, 6], 135)
                                     ), axis=-1
                                    )
                           ),
            Polarization=(['across', 'along','Antenna'],
                          np.stack((np.full([5, 6], 'VV'),
                                    np.full([5, 6], 'VV'),
                                    np.full([5, 6], 'HH'),
                                    np.full([5, 6], 'VV')
                                    ), axis=-1
                                   )
                          ),
#             Sigma0=( ['across','along','Antenna'], np.full([9,11,4], 1.01) ),
#             dsig0=( ['across','along','Antenna'], np.full([9,11,4], 0.05) ),
#             RVL=( ['across','along','Antenna'], np.full([9,11,4], 0.5) ),
#             drvl=( ['across','along','Antenna'], np.full([9,11,4], 0.01) ),
        ),
    coords=dict(
            across=np.arange(0,5),
            along=np.arange(0,6),
            Antenna=['Fore','MidV','MidH','Aft'],
        ),
)
level1 = level1.set_coords([
    'CentralWavenumber',
    'CentralFreq',
    'IncidenceAngleImage',
    'AntennaAzimuthImage',
    'Polarization', 
])


In [4]:
geo = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['across', 'along'], np.full([5, 6], 10)),
            WindDirection=(['across', 'along'], np.full([5, 6], 150)),
            CurrentVelocity=(['across', 'along'], np.full([5, 6], 1)),
            CurrentDirection=(['across', 'along'], np.full([5, 6], 150)),
        ),
        coords=dict(
            across=np.arange(0, 5),
            along=np.arange(0, 6),
        ),
    )
geo

<xarray.Dataset>
Dimensions:           (across: 5, along: 6)
Coordinates:
  * across            (across) int64 0 1 2 3 4
  * along             (along) int64 0 1 2 3 4 5
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150

In [5]:
level1

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    *empty*

In [6]:
gmf=dotdict({'nrcs': dotdict({'name': 'nscat4ds'})})
gmf['doppler'] = dotdict({'name': 'mouche12'})

In [7]:
level1['Sigma0'] = ss.gmfs.nrcs.compute_nrcs(level1, geo, gmf.nrcs)*1.001

In [8]:
model_rvl_list = [None] * level1.Antenna.size
model_wasv_list = [None] * level1.Antenna.size
for aa, ant in enumerate(level1.Antenna.data):
    model_wasv_list[aa] = ss.gmfs.doppler.compute_wasv(level1.sel(Antenna=ant), geo, gmf=gmf.doppler.name)
    model_rvl_list[aa] = ss.gmfs.doppler.compute_total_surface_motion(level1.sel(Antenna=ant), geo, gmf=gmf.doppler.name)
level1['WASV'] = xr.concat(model_wasv_list, dim='Antenna')*1.001
level1['RSV'] = xr.concat(model_rvl_list, dim='Antenna')*1.001

In [9]:
# Add NaN for RVL for the mid antennas
level1.RSV[1,:,:] = np.full([5,6], np.nan)
level1.RSV[2,:,:] = np.full([5,6], np.nan)


In [10]:
model_rvl_list = [None] * level1.Antenna.size
for aa, ant in enumerate(level1.Antenna.data):
    model_rvl_list[aa] = ss.gmfs.doppler.compute_total_surface_motion(level1.sel(Antenna=ant), geo, gmf='mouche12')

model = level1.drop_vars([var for var in level1.data_vars])
model['RSV'] = xr.concat(model_rvl_list, dim='Antenna')
model['Sigma0'] = ss.gmfs.nrcs.compute_nrcs(level1, geo, gmf=gmf.nrcs)

In [11]:
noise = level1.drop_vars([var for var in level1.data_vars])
# noise = level1.drop_vars(['Sigma0','dsig0','RVL','drvl'])
noise['Sigma0'] = level1.Sigma0*0.05
noise['RSV'] = level1.RSV*0.05

In [12]:
res = (model-level1)/noise

In [13]:
np.concatenate(
    (res.Sigma0.sel(across=1, along=1).data, res.RSV.sel(across=1, along=1).data)
)
    

array([-0.01998002, -0.01998002, -0.01998002, -0.01998002, -0.01998002,
               nan,         nan, -0.01998002])

# Test Fun_residual

In [14]:
sl1 = level1.sel(across=1, along=1)
sn = noise.sel(across=1, along=1)

In [15]:
ss.retrieval.cost_function.fun_residual([5,5,1,1], sl1, sn, gmf)

array([   9.01895349,   12.1681688 ,   12.60884007,   16.59575996,
       -691.66077601,    0.        ,    0.        ,   17.09819907])

### test fun_residual with 2D fields

In [16]:
[geo['U'], geo['V']] = ss.utils.tools.windSpeedDir2UV(geo.WindSpeed, geo.WindDirection)
[geo['C_U'], geo['C_V']] = ss.utils.tools.currentVelDir2UV(geo.CurrentVelocity, geo.CurrentDirection)

In [17]:
geo

<xarray.Dataset>
Dimensions:           (across: 5, along: 6)
Coordinates:
  * across            (across) int64 0 1 2 3 4
  * along             (along) int64 0 1 2 3 4 5
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150
    U                 (across, along) float64 -5.0 -5.0 -5.0 ... -5.0 -5.0 -5.0
    V                 (across, along) float64 8.66 8.66 8.66 ... 8.66 8.66 8.66
    C_U               (across, along) float64 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5
    C_V               (across, along) float64 -0.866 -0.866 ... -0.866 -0.866

In [18]:
level1

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    WASV                 (Antenna, across, along) float64 0.3227 ... -1.498
    RSV                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [19]:
results = ss.retrieval.cost_function.fun_residual([geo.U.data, geo.V.data, geo.C_U.data, geo.C_V.data], level1, noise, gmf)

In [20]:
results = ss.retrieval.cost_function.fun_residual([geo.U, geo.V, geo.C_U, geo.C_V], level1, noise, gmf)

In [21]:
results

array([[[-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857]],

       [[-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737]],

       [[-3.27504276, -3.2

In [22]:
ss.retrieval.cost_function.fun_residual([geo.U, geo.V, geo.C_U, geo.C_V], level1, noise, gmf).shape

(8, 5, 6)

# Test least_squares with fun_residual

In [23]:
opt = {
        'method': 'trf', # Trust Region Reflective algorithm, particularly suitable for large sparse problems with bounds. Generally robust method.
        'xtol':  1e-1, # Tolerance for termination by the change of the independent variables
        'x_scale': [7,7,.5,.5], # Characteristic scale of each variable.
        'bounds': ([-30,-30,-5,-5], [30,30,5,5]),
    }
x0_variables = [4,4,0,0]

In [24]:
lmout = least_squares(
    ss.retrieval.cost_function.fun_residual,
    x0_variables,
    args=(sl1, sn, gmf),
    **opt
)

In [25]:
lmout

 active_mask: array([0, 0, 0, 0])
        cost: 359.2870934440441
         fun: array([-5.84578029,  0.84953228,  3.50291906, 13.55825463,  7.28518626,
        0.        ,  0.        , 20.84487192])
        grad: array([-1.11574674e-01,  1.29430683e+01, -1.06386311e+03, -2.18671347e+03])
         jac: array([[-5.87770054e+00, -3.71198733e+00,  5.87770021e+00,
         3.71198744e+00],
       [-4.73286728e+00, -1.55214232e+00,  4.73286674e+00,
         1.55214188e+00],
       [-3.78176642e+00, -1.51455966e+00,  3.78176609e+00,
         1.51455951e+00],
       [-1.26884883e+00, -1.57278294e-01,  1.26884854e+00,
         1.57278061e-01],
       [ 0.00000000e+00,  0.00000000e+00, -2.22214298e+02,
        -2.22214293e+02],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.66203537e+01,
        -2.66203547e+01]])
     message: '`

# Test find_minima

In [26]:
level1

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    WASV                 (Antenna, across, along) float64 0.3227 ... -1.498
    RSV                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [27]:
L1stack = level1.stack(z=("across", "along"))

In [28]:
noise

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.003564 ... 0.008339
    RSV                  (Antenna, across, along) float64 0.003182 ... -0.02656

In [29]:
Nstack = noise.stack(z=("across", "along"))

In [30]:
gmf

{'nrcs': {'name': 'nscat4ds'}, 'doppler': {'name': 'mouche12'}}

In [31]:
lmoutmap = [None] * L1stack.z.size

In [30]:
for ii, zindex in enumerate(L1stack.z.data):
    print(ii)
    sL1 = L1stack.sel(z=zindex)
    sN = Nstack.sel(z=zindex)
    lmoutmap[ii] = ss.retrieval.cost_function.find_minima(sL1, sN, gmf)

0


KeyboardInterrupt: 

In [31]:
lmmap = xr.concat(lmoutmap, dim='z')

TypeError: can only concatenate xarray Dataset and DataArray objects, got <class 'NoneType'>

In [32]:
lmmap['z'] = L1stack.z

NameError: name 'lmmap' is not defined

In [33]:
sol = lmmap.unstack(dim='z')

NameError: name 'lmmap' is not defined

In [34]:
sol

NameError: name 'sol' is not defined

In [35]:
s1 = sol.isel(along=0, across=0).sortby('cost')

NameError: name 'sol' is not defined

In [36]:
s1.x.isel(Ambiguities=0).sel(x_variables='c_v')

NameError: name 's1' is not defined

# Test wind_current_retrieval

In [32]:
sL1 = level1.isel(along=0, across=0)
sN = noise.isel(along=0, across=0) # Pass wind_current_retrieval
ambiguity = {'name': 'sort_by_cost'}

In [33]:
l2 = ss.retrieval.level2.wind_current_retrieval(sL1, sN, gmf, ambiguity) # Pass

To Be Done find_initial_value
To be done


In [31]:
sL1 = level1.isel(along=slice(0,2), across=slice(0,2))
sN = noise.isel(along=slice(0,2), across=slice(0,2))  # Pass wind_current_retrieval

In [32]:
l2 = ss.retrieval.level2.wind_current_retrieval(sL1, sN, gmf, ambiguity)

To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
> /Users/admartin/OneDrive/OneDrive - NOC/Documents/NOC-OneDrive/workLocal/SEASTAR/seastar/seastar/retrieval/level2.py(83)wind_current_retrieval()
     81     pdb.set_trace()
     82     # level2 = xr.Dataset()
---> 83     level2['x'] = sol.x.isel(Ambiguities=0)
     84     level2['CurrentU'] = level2.x.sel(x_variables='c_u')
     85     level2['CurrentV'] = level2.x.sel(x_variables='c_v')

ipdb> continue
To be done


In [32]:
sL1 = level1.isel(along=0, across=slice(0,2))
sN = noise.isel(along=0, across=slice(0,2)) # Fail wind_current_retrieval

In [33]:
l2 = ss.retrieval.level2.wind_current_retrieval(sL1, sN, gmf, ambiguity)

To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To be done


In [33]:
l2

<xarray.Dataset>
Dimensions:              (across: 2, along: 2, Antenna: 4, x_variables: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1
  * along                (along) int64 0 1
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
  * x_variables          (x_variables) <U3 'u' 'v' 'c_u' 'c_v'
Data variables:
    x                    (x_variables, across, along) float64 7.118 ... 0.008477
    CurrentU             (across, along) float64 -1.337 0.4997 0.9868 1.014
    CurrentV             (across, along) float64 0.2976 -0.8658 0.01463 0.008477
    WindU                (across, along) float64 7.118 -4.503 -6.49 -6.499
    WindV                (across, along) float64 0.3392 7.8 3.186 3.103
    CurrentVelocity      (across, along) float64 1.37 0.9997 0.9869 1.014
    CurrentDirection     (across, along) float64 282.6 150.0 89.15 89.52
    WindSpeed            (across, along) float64 7.126 9.006 7.23 7.202
    WindDirection        (across, along) float64 267.3 150.0 116.1 115.5

In [45]:
l2.x[3,:,:]

<xarray.DataArray 'x' (across: 2, along: 2)>
array([[-0.86595479, -0.86598589],
       [-0.90217959,  0.00475549]])
Coordinates:
  * across             (across) int64 0 1
  * along              (along) int64 0 1
    x_variables        <U3 'c_v'
    CentralFreq        float64 1.35e+10
    CentralWavenumber  int64 270
Attributes:
    description:  Solution found

In [41]:
lmout

<xarray.Dataset>
Dimensions:              (across: 2, along: 2, x_variables: 4, Observables: 2,
                          Antenna: 4, Ambiguities: 4, fun_variables: 8,
                          extrema: 2)
Coordinates:
  * across               (across) int64 0 1
  * along                (along) int64 0 1
  * x_variables          (x_variables) <U3 'u' 'v' 'c_u' 'c_v'
  * Observables          (Observables) <U6 'sigma0' 'RVL'
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
    fun_variables        (Observables, Antenna) int64 0 1 2 3 4 5 6 7
    CentralFreq          float64 1.35e+10
    CentralWavenumber    int64 270
    AntennaAzimuthImage  (Antenna) int64 45 90 90 135
    Polarization         (Antenna) <U2 'VV' 'VV' 'HH' 'VV'
    IncidenceAngleImage  (Antenna) int64 30 30 30 30
Dimensions without coordinates: Ambiguities, extrema
Data variables: (12/17)
    x                    (Ambiguities, x_variables, across, along) float64 -4...
    active_mask          (Ambiguities, x_variables, across, along) int64 0 ... 0
    grad                 (Ambiguities, x_variables, across, along) float64 -1...
    fun                  (Ambiguities, Observables, Antenna, across, along) float64 ...
    jac                  (Ambiguities, fun_variables, x_variables, across, along) float64 ...
    x0                   (Ambiguities, x_variables, across, along) float64 -7...
    ...                   ...
    message              (Ambiguities, across, along) <U42 '`xtol` terminatio...
    success              (Ambiguities, across, along) bool True True ... True
    method               (across, along) <U3 'trf' 'trf' 'trf' 'trf'
    xtol                 (across, along) float64 0.001 0.001 0.001 0.001
    x_scale              (x_variables, across, along) float64 7.0 7.0 ... 0.5
    bounds               (extrema, x_variables, across, along) int64 -30 ... 5

In [62]:
stacked

<xarray.Dataset>
Dimensions:  (z: 2)
Coordinates:
    along    int64 0
  * z        (z) object MultiIndex
  * across   (z) int64 0 1
Data variables:
    mydata   (z) int64 0 2

In [89]:
ds = xr.Dataset(
    data_vars=dict(
        mydata = ( ['across', 'along'], [[0,1],[2,3]])
    ),
    coords=dict(# <- changed 
        across=[0,1],# <- changed 
        along=[0,1],# <- changed 
    ),    # <- changed 
)
stacked = ds.isel(along=0).stack(z=["across"]) # <- changed
newlist = [None] * stacked.z.size
for ii, zindex in enumerate(stacked.z.data):
    newlist[ii] = stacked.mydata.sel(z=zindex)
    

In [91]:
newds = xr.concat(newlist, dim='z')
newds = newds.set_index(z=['across']) # <- changed
newds.unstack(dim='z') # Fail with ValueError: cannot unstack dimensions that do not have exactly one multi-index: ('z',)

ValueError: cannot unstack dimensions that do not have exactly one multi-index: ('z',)

In [66]:
newds = xr.concat(newlist, dim='z')
newds

<xarray.DataArray 'mydata' (z: 2)>
array([0, 2])
Coordinates:
    along    int64 0
  * z        (z) object (0,) (1,)
    across   (z) int64 0 1

In [68]:
newds = newds.set_index(z=['across',])
newds

<xarray.DataArray 'mydata' (z: 2)>
array([0, 2])
Coordinates:
    along    int64 0
  * z        (z) int64 0 1

In [69]:
newds.unstack(dim='z')

ValueError: cannot unstack dimensions that do not have exactly one multi-index: ('z',)

In [49]:
a = ds.isel(along=0)

In [56]:
a.stack(z=['across'])

<xarray.Dataset>
Dimensions:  (z: 2)
Coordinates:
    along    int64 0
  * z        (z) object MultiIndex
  * across   (z) int64 0 1
Data variables:
    mydata   (z) int64 0 2

In [76]:
newds.set_index(z=['across'])

<xarray.DataArray 'mydata' (z: 2)>
array([0, 2])
Coordinates:
    along    int64 0
  * z        (z) int64 0 1

In [88]:
import pandas as pd
tuples = [(0,),(1,)],
index = pd.MultiIndex.from_tuples(tuples, names=["z","across"])
index

MultiIndex([((0,), (1,))],
           names=['z', 'across'])

In [82]:
index

MultiIndex([((0,), (1,))],
           names=['z', 'across'])

In [86]:
newds.set_index(z=['across',], index_cls=index)

TypeError: unhashable type: 'MultiIndex'